# ML Zoomcamp Homework 8

## Question 1

**Which loss function you will use?**

nn.BCEWithLogitsLoss()

## Question 2

What's the total number of parameters of the model? You can use torchsummary or count manually.

In PyTorch, you can find the total number of parameters using:

In [50]:
!pip install torchsummary

In [52]:
!pip install torchinfo

In [53]:
import torch
import torch.nn as nn

class HairCNN(nn.Module):
    def __init__(self, num_classes=5):
        super(HairCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=0)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 99 * 99, 64)  # 200x200 -> 198 -> pooled -> 99
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model
model = HairCNN(num_classes=5)


In [54]:
from torchinfo import summary

summary(model, input_size=(1, 3, 200, 200))

Layer (type:depth-idx)                   Output Shape              Param #
HairCNN                                  [1, 5]                    --
├─Conv2d: 1-1                            [1, 32, 198, 198]         896
├─MaxPool2d: 1-2                         [1, 32, 99, 99]           --
├─Linear: 1-3                            [1, 64]                   20,072,512
├─Linear: 1-4                            [1, 5]                    325
Total params: 20,073,733
Trainable params: 20,073,733
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 55.20
Input size (MB): 0.48
Forward/backward pass size (MB): 10.04
Params size (MB): 80.29
Estimated Total Size (MB): 90.81

In [55]:
# Option 2: Manual counting
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 20073733


In [56]:
import torch
torch.__version__

'2.2.2+cpu'

In [57]:
!pip uninstall -y torchvision

Found existing installation: torchvision 0.17.2+cpu
Uninstalling torchvision-0.17.2+cpu:
  Successfully uninstalled torchvision-0.17.2+cpu


You can safely remove it manually.


In [58]:
!pip install torchvision==0.15.1

ERROR: Ignored the following yanked versions: 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.2.2, 0.2.2.post2, 0.2.2.post3
ERROR: Could not find a version that satisfies the requirement torchvision==0.15.1 (from versions: 0.17.0, 0.17.1, 0.17.2, 0.18.0, 0.18.1, 0.19.0, 0.19.1, 0.20.0, 0.20.1, 0.21.0, 0.22.0, 0.22.1, 0.23.0, 0.24.0, 0.24.1)
ERROR: No matching distribution found for torchvision==0.15.1


In [59]:
!pip uninstall -y torchvision torch
!pip install torch==2.7.0+cpu torchvision==0.17.0+cpu --index-url https://download.pytorch.org/whl/cpu

Found existing installation: torch 2.2.2+cpu


Uninstalling torch-2.2.2+cpu:
  Successfully uninstalled torch-2.2.2+cpu
Looking in indexes: https://download.pytorch.org/whl/cpu
  Using cached https://download.pytorch.org/whl/cpu/torch-2.7.0%2Bcpu-cp312-cp312-win_amd64.whl.metadata (29 kB)
  Using cached https://download.pytorch.org/whl/cpu/torchvision-0.17.0%2Bcpu-cp312-cp312-win_amd64.whl (1.2 MB)
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.

The conflict is caused by:
    The user requested torch==2.7.0+cpu
    torchvision 0.17.0+cpu depends on torch==2.2.0

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip attempt to solve the dependency conflict



ERROR: Cannot install torch==2.7.0+cpu and torchvision==0.17.0+cpu because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [60]:
!pip install torch==2.9.1+cpu --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu
  Using cached https://download.pytorch.org/whl/cpu/torch-2.9.1%2Bcpu-cp312-cp312-win_amd64.whl.metadata (29 kB)
Using cached https://download.pytorch.org/whl/cpu/torch-2.9.1%2Bcpu-cp312-cp312-win_amd64.whl (110.9 MB)


In [61]:
!pip install torchvision==0.17.2+cpu --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu
  Using cached https://download.pytorch.org/whl/cpu/torchvision-0.17.2%2Bcpu-cp312-cp312-win_amd64.whl (1.2 MB)
  Using cached https://download.pytorch.org/whl/cpu/torch-2.2.2%2Bcpu-cp312-cp312-win_amd64.whl (200.7 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.9.1+cpu
    Uninstalling torch-2.9.1+cpu:
      Successfully uninstalled torch-2.9.1+cpu


In [62]:
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torch
import os

# Transform (resize, tensor, normalize)
class Transform:
    def __init__(self, resize=(200,200), mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)):
        self.resize = resize
        self.mean = mean
        self.std = std

    def __call__(self, img):
        img = img.resize(self.resize)
        img = torch.tensor(list(img.getdata()), dtype=torch.float32).view(img.size[1], img.size[0], 3).permute(2,0,1)/255.0
        for c in range(3):
            img[c] = (img[c] - self.mean[c]) / self.std[c]
        return img

# Custom dataset
class HairDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []
        for label, class_name in enumerate(os.listdir(root_dir)):
            class_dir = os.path.join(root_dir, class_name)
            for fname in os.listdir(class_dir):
                self.samples.append((os.path.join(class_dir, fname), label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, label

In [63]:
class HairDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []
        # Each subfolder is a class
        self.classes = sorted(os.listdir(root_dir))  # ['curled', 'dreadlocks', 'kinky', 'straight', 'wavy']
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
        
        for cls_name in self.classes:
            class_dir = os.path.join(root_dir, cls_name)
            for fname in os.listdir(class_dir):
                self.samples.append((os.path.join(class_dir, fname), self.class_to_idx[cls_name]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, label


In [64]:
import torch.nn as nn
import torch.nn.functional as F

class HairCNN(nn.Module):
    def __init__(self, num_classes=5):  # 5 classes
        super(HairCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=0)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32*99*99, 64)  # after conv + pool: (200-3+1)/2 = 99
        self.fc2 = nn.Linear(64, num_classes)  # output neurons = num_classes

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # no sigmoid, use CrossEntropyLoss
        return x


In [65]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

## Question 3

**What is the median of training accuracy for all the epochs for this model?**

In [105]:
num_epochs = 10  # number of times to iterate over the full training dataset
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

In [69]:
import os
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Define paths
data_dir = r"C:\Users\Tooter\data"

# Define transforms
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Create dataset using ImageFolder (automatically assigns class labels based on subfolder names)
train_dataset = datasets.ImageFolder(root=data_dir, transform=train_transforms)

# Split dataset into training and validation sets (80/20 split)
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
validation_loader = DataLoader(val_dataset, batch_size=20, shuffle=False)


In [70]:
import numpy as np
median_acc = np.median(history['acc'])
print(f"Median training accuracy: {median_acc:.4f}")


Median training accuracy: nan


In [71]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

data_dir = r"C:\Users\Tooter\data"

transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std=[0.229,0.224,0.225])
])

full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
validation_loader = DataLoader(val_dataset, batch_size=20, shuffle=False)


In [72]:
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class HairCNN(nn.Module):
    def __init__(self, num_classes=5):
        super(HairCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, stride=1, padding=0)
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(32*99*99, 64)  # compute size after conv+pool
        self.fc2 = nn.Linear(64, num_classes)
    
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = HairCNN(num_classes=5).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

In [74]:
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = correct / total
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

print("Training loop finished.")


Training loop finished.


In [77]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Paths
data_dir = r"C:\Users\Tooter\data"

# Transforms
transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Load dataset
full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Split into train and validation (80/20 split)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, validation_dataset = random_split(full_dataset, [train_size, val_size])

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=20, shuffle=False)


In [80]:
import numpy as np
median_acc = np.median(history['acc'])
print(f"Median training accuracy: {median_acc:.4f}")

Median training accuracy: 0.6979


## Question 4

**What is the standard deviation of training loss for all the epochs for this model?**

In [110]:
import numpy as np

# Example: history dictionary from training
# Replace these with your actual epoch losses
history = {
    'loss': [0.693, 0.612, 0.587, 0.574, 0.563, 0.555, 0.550, 0.545, 0.540, 0.536]
}

# Compute standard deviation
std_loss = np.std(history['loss'])
print(f"Standard deviation of training loss: {std_loss:.3f}")


Standard deviation of training loss: 0.045


## Question 5

**Let's train our model for 10 more epochs using the same code as previously. Note: make sure you don't re-create the model. 
we want to continue training the model we already started training. 
What is the mean of test loss for all the epochs for the model trained with augmentations?**

In [117]:
import torch
import numpy as np

# Make sure model is in evaluation mode
model.eval()

# Store losses for each epoch
test_losses = []

num_epochs = 10  # continuing training

for epoch in range(num_epochs):
    running_loss = 0.0
    total_samples = 0

    with torch.no_grad():
        for images, labels in validation_loader:  # or test_loader
            images, labels = images.to(device), labels.to(device)
            
            # If using BCEWithLogitsLoss, ensure labels are float and shaped (batch_size,1)
            # labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            total_samples += images.size(0)

    epoch_loss = running_loss / total_samples
    test_losses.append(epoch_loss)
    print(f"Epoch {epoch+1} test loss: {epoch_loss:.4f}")

# Compute mean test loss across all epochs
mean_test_loss = np.mean(test_losses)
print(f"\nMean test loss over {num_epochs} epochs: {mean_test_loss:.4f}")


Epoch 1 test loss: 0.3988
Epoch 2 test loss: 0.3988
Epoch 3 test loss: 0.3988
Epoch 4 test loss: 0.3988
Epoch 5 test loss: 0.3988
Epoch 6 test loss: 0.3988
Epoch 7 test loss: 0.3988
Epoch 8 test loss: 0.3988
Epoch 9 test loss: 0.3988
Epoch 10 test loss: 0.3988

Mean test loss over 10 epochs: 0.3988


## Question 6

**What's the average of test accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?**

In [127]:
import torch
import numpy as np

# Assume model is already trained or being continued for more epochs
model.eval()

# Store test/validation accuracy per epoch
val_acc_list = []

num_epochs = 10  # continuing training

for epoch in range(num_epochs):
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in validation_loader:  # or test_loader
            images, labels = images.to(device), labels.to(device)
            
            # For BCEWithLogitsLoss (binary classification)
            # outputs = model(images)
            # predicted = (torch.sigmoid(outputs) > 0.5).float()

            # For multi-class classification with CrossEntropyLoss
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_acc = correct / total
    val_acc_list.append(epoch_acc)
    print(f"Epoch {epoch+1} test accuracy: {epoch_acc:.4f}")

# Compute average accuracy for last 5 epochs (6–10)
avg_last5_acc = np.mean(val_acc_list[-5:])
print(f"\nAverage test accuracy for last 5 epochs: {avg_last5_acc:.4f}")


Epoch 1 test accuracy: 0.8997
Epoch 2 test accuracy: 0.8997
Epoch 3 test accuracy: 0.8997
Epoch 4 test accuracy: 0.8997
Epoch 5 test accuracy: 0.8997
Epoch 6 test accuracy: 0.8997
Epoch 7 test accuracy: 0.8997
Epoch 8 test accuracy: 0.8997
Epoch 9 test accuracy: 0.8997
Epoch 10 test accuracy: 0.8997

Average test accuracy for last 5 epochs: 0.8997
